# How to Get Fundamental Stock Data in Python for Free

#### Alpha Vantage offers a free API for enterprise-grade financial market data 

### Intro

This post is a solution to a problem I had, and I am sharing it in the hopes it makes stock analysis more accessible for retail investors and traders. While trying to find data for analyzing stocks I ran into an issue, specifically one of cost. I was searching for APIs with fundamental data, that is the data that comes from the underlying business(revenue, earnings, cash flow) of a stock rather than the technical performance of a stock(price, volume). The common problem I ran into was that most APIs I came across gated the fundamental data behind either a paywall or the amount of data was cut down(typically 2 years or less). Not wanting to pay for data(which can get quite expensive), I was searching for an API that had fundamental data for free, which also provided a decent amount of history. Enter Alpha Vantage.

### What is Alpha Vantage?

[Alpha Vantage](https://www.alphavantage.co/#page-top) is an API that offers some of the most complete and accessible data I have seen for free. So what's the catch you may ask? The data is limited by the amount and time between API calls. The free option at the time of writing allows for 500 calls daily and a maximum of 5 per minute. Not too bad for free. But for heavier applications, this likely won't be sufficient, in which case they do also offer paid tiers(all with unlimited daily calls) ranging from 75 calls per minute to 1200 per minute (costing \\$50 and \\$250  respectively)with other options in between.

### Useful Info

In this post, I will be covering the Python API but they also support a wide range of other languages and even have built-in support for Excel and Google Sheets. I'd highly recommend checking out their [documentation](https://www.alphavantage.co/documentation/) and their [tutorials](https://www.alphavantage.co/academy/) if you want to learn more. Finally, I will also be utilizing this great library, [alpha_vantage](https://github.com/RomelTorres/alpha_vantage), which is a wrapper for the API written by Romel Torres. It simplifies the API calls and allows for the conversion of the JSON to a Pandas [dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). This will just help cut out one more step.  But now let's dive in.

The first step is to import libraries that will be used, as well as our API key. The main libraries used will be the pandas and alpha_vantage libraries. I will just be using the fundamental data, but the library supports all of the types of data/API calls Alpha Vantage offers such as Time Series, Forex, Crypto and more. My API key is in a text file so that it isn't visible here, but feel free to replace my line with just the string of your key. Be careful not to share this with anyone as they could then use your key leaving you unable to make API calls. 

In [1]:
from alpha_vantage.fundamentaldata import FundamentalData
import json
import pandas as pd
import requests
import time
apikey = pd.read_csv('apikey.txt', header=None)[0][0]

Now let's try to call the API for the first time without the wrapper library to help us out. As you can see a JSON is returned. This looks fairly tidy, but let's see how that compares to the alpha_vantage helper output.

In [3]:
#Stock overview data for SQ(Square)
ticker = 'SQ'
url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol='+ticker+'&apikey='+apikey
r = requests.get(url)
data = r.json()

print(data)

{'Symbol': 'SQ', 'AssetType': 'Common Stock', 'Name': 'Square Inc', 'Description': 'Square, Inc. creates tools that allow merchants to accept card payments; they also provide reports and analysis, and next day settlement. The company is headquartered in San Francisco, California.', 'CIK': '1512673', 'Exchange': 'NYSE', 'Currency': 'USD', 'Country': 'USA', 'Sector': 'TECHNOLOGY', 'Industry': 'SERVICES-PREPACKAGED SOFTWARE', 'Address': '1455 MARKET STREET, SUITE 600, SAN FRANCISCO, CA, US', 'FiscalYearEnd': 'December', 'LatestQuarter': '2021-06-30', 'MarketCapitalization': '113398292000', 'EBITDA': '433271000', 'PERatio': '222.64', 'PEGRatio': '2.391', 'BookValue': '5.87', 'DividendPerShare': 'None', 'DividendYield': '0', 'EPS': '1.108', 'RevenuePerShareTTM': '35.25', 'ProfitMargin': '0.036', 'OperatingMarginTTM': '0.0221', 'ReturnOnAssetsTTM': '0.0204', 'ReturnOnEquityTTM': '0.246', 'RevenueTTM': '15930786000', 'GrossProfitTTM': '2744583000', 'DilutedEPSTTM': '1.108', 'QuarterlyEarnings

Below we supply the same information as before with key and symbol for the stock we want but there are two differences. The first difference is that we call a specific method based on the data group we want, in this case, the company overview. We can also now specify the output format for the data. I will use pandas as this parameter will return the data in the form of a dataframe but also supports JSON(default) and CSV(with some exceptions). As you can see the dataframe nicely formats all of the data, similar to what one might find in a relational database or an Excel sheet. Now we can also start to see all of the different data which Alpha Vantage provides. The data I will eventually be using breaks down to these four groups, which once again you can find more information in the [documentation](https://www.alphavantage.co/documentation/) but I'll provide a brief description.
1. Company Overview - Provides key metrics and information about a company
2. Income Statement(Annual) - The annual income statement
3. Balance Sheet(Annual) - The annual balance sheet data 
4. Cash Flow(Annual) - The annual cash flow statement 

In [4]:
fd = FundamentalData(key = apikey, output_format='pandas')
api_data, api_meta = fd.get_company_overview(symbol = ticker)
api_data.head(1)

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,ShortPercentFloat,PercentInsiders,PercentInstitutions,ForwardAnnualDividendRate,ForwardAnnualDividendYield,PayoutRatio,DividendDate,ExDividendDate,LastSplitFactor,LastSplitDate
NaN,SQ,Common Stock,Square Inc,"Square, Inc. creates tools that allow merchant...",1512673,NYSE,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,...,0.093,1.204,74.7,0,0,0,None,None,None,None


### Creating a Row with Desired Columns

Now that we can easily call the data for a single company it would be really useful if we could repeat that process for multiple companies. That way we can append each row and our table will be made up of a list of companies with current data. From there we can decide what we want to do with the data, whether it's doing analyses or training models. In order to get the data in a repeatable way, I think it would be useful to make a helper function that extracts any relevant columns from each table we want. Below I implemented a function that takes in each relevant table's data, subsets the columns that I want(it also calculates the 3-year revenue growth rate), and adds that to a single row.  

In [5]:
#Define function calc_row. Take in 4 tables(returned from api call), and create final row

def calc_row(overview_val, income_val, balance_val, cashflow_val):
    #Add Overview Values and reset index(its off for some reason)
    row_data = overview_val[['Symbol','Sector','PercentInsiders','PercentInstitutions','MarketCapitalization',
                             'RevenueTTM','ReturnOnAssetsTTM','EBITDA','ProfitMargin','OperatingMarginTTM',
                             'GrossProfitTTM','ForwardAnnualDividendYield']]
    row_data = row_data.reset_index(drop=True)
    
    #Add Income (calculate 3yr revenue growth rate or approximation/None)
    rev_growth_3y = 0
    try:
        if(len(income['totalRevenue']) >= 4):
            rev_growth_3y = (int(income['totalRevenue'][0]) - int(income['totalRevenue'][3])) / int(income['totalRevenue'][3])
        else:
            rev_growth_3y = overview['QuarterlyRevenueGrowthYOY'] * 3
    except:
        rev_growth_3y = None
   
    row_data['RevenueGrowth3Yr'] = rev_growth_3y
        
    #Add Balance values, have to get matrix in correct format
    balance_cols = ['totalAssets','totalLiabilities','cashAndShortTermInvestments']
    row_data[balance_cols] = balance_val[balance_cols].iloc[0:1].reset_index(drop=True)
    
     
    #Add Cashflow
    row_data[['operatingCashflow','capitalExpenditures']] = cashflow_val[['operatingCashflow','capitalExpenditures']].iloc[0:1].reset_index(drop=True)
    
    return row_data

Now to test if this function works as intended. To do this I call the API on the four tables I want. I then supply that data to the function, add the returned row to the test table, and display the table. As you can see all information appears to be correct but some additional formatting may be desired in the future, such as converting the data types to floating-point or integers. But for now, this looks sufficient.

In [7]:
#Call api 4 times for each relevant table. Split by data, metadata
overview, o_meta = fd.get_company_overview(symbol = ticker)
income, i_meta = fd.get_income_statement_annual(symbol = ticker)
balance, b_meta = fd.get_balance_sheet_annual(symbol = ticker)
cashflow, c_meta = fd.get_cash_flow_annual(symbol = ticker)

In [9]:
#Create dataframe, calculate/create each row using helper function and concating to dataframe 
test_table = pd.DataFrame()
company_row = calc_row(overview, income, balance, cashflow)
test_table = pd.concat([test_table, company_row], ignore_index=True)
test_table

,Symbol,Sector,PercentInsiders,PercentInstitutions,MarketCapitalization,RevenueTTM,ReturnOnAssetsTTM,EBITDA,ProfitMargin,OperatingMarginTTM,GrossProfitTTM,ForwardAnnualDividendYield,RevenueGrowth3Yr,totalAssets,totalLiabilities,cashAndShortTermInvestments,operatingCashflow,capitalExpenditures
0,SQ,TECHNOLOGY,1.204,74.7,113398292000,15930786000,0.0204,433271000,0.036,0.0221,2744583000,0,3.289292,9869550000,7187981000,4229428000,381603000,138402000


In [10]:
test_table.dtypes

Symbol                          object
Sector                          object
PercentInsiders                 object
PercentInstitutions             object
MarketCapitalization            object
RevenueTTM                      object
ReturnOnAssetsTTM               object
EBITDA                          object
ProfitMargin                    object
OperatingMarginTTM              object
GrossProfitTTM                  object
ForwardAnnualDividendYield      object
RevenueGrowth3Yr               float64
totalAssets                     object
totalLiabilities                object
cashAndShortTermInvestments     object
operatingCashflow               object
capitalExpenditures             object
dtype: object

### Making a Table with Multiple Companies

Now that we have a method for creating a row with the data we want, the final part should be looping through a list of companies whose data we want and combining it all into a single table. To do that I'll import a CSV which is a list of approximately 100 companies I want to analyze. 

In [11]:
#Import and display the list of companies whose data we want
tickers = pd.read_csv("Companies100.csv")
tickers

,Companies
0,DKNG
1,WBA
2,AAL
3,AMZN
4,SQ
...,...
95,BRK/B
96,EDIT
97,TWST
98,SGFY


Due to the constraints of the free API, I will end up subsetting the list to just the first 15 companies to save time, but if you wish to do more that is also possible. As a reminder, the restrictions for the free version are 500 calls per day, and 5 calls per minute. But then the rest is just implementing a for loop which iterates through each company, calculates a row for each, and appends that row to the table. After that, I sleep the loop for 65 seconds and continue to the next company. 

In [12]:
# Create a final dataframe. 
#Subset data(for time and to limit API calls, which happens 4x per loop) 
final_table = pd.DataFrame()
tickers_slim = tickers[0:15]


#Loop through each company, call API 4 times for relevant data, create row, add it to the table.
#Finally we need to pause for 60+ seconds to make sure the API doesn't stop the loop(max 5 calls/min)
for ticker in tickers_slim['Companies']:
    print(ticker)
    
    overview, o_meta = fd.get_company_overview(symbol = ticker)
    income, i_meta = fd.get_income_statement_annual(symbol = ticker)
    balance, b_meta = fd.get_balance_sheet_quarterly(symbol = ticker)
    cashflow, c_meta = fd.get_cash_flow_quarterly(symbol = ticker)
    
    company_row = calc_row(overview, income, balance, cashflow)
    final_table = pd.concat([final_table, company_row], ignore_index=True)
    time.sleep(65) 

DKNG
WBA
AAL
AMZN
SQ
TSLA
NSTG
CRSP
NKE
AAPL
TER
MMM
HON
BABA
GOOGL


The last task is to save and load the table so that the loop is not required in the future. This can be a checkpoint unless additional companies' data is desired. As you can see when displaying the final table all of the companies with relevant data are displayed.

In [13]:
#Save data so we don't need to run loop again
final_table.to_csv('final_table.csv', index=False)

In [14]:
#Load data for viewing
final_table = pd.read_csv('final_table.csv',)
final_table

,Symbol,Sector,PercentInsiders,PercentInstitutions,MarketCapitalization,RevenueTTM,ReturnOnAssetsTTM,EBITDA,ProfitMargin,OperatingMarginTTM,GrossProfitTTM,ForwardAnnualDividendYield,RevenueGrowth3Yr,totalAssets,totalLiabilities,cashAndShortTermInvestments,operatingCashflow,capitalExpenditures
0,DKNG,TRADE & SERVICES,7.720,60.89,24226703000,1064940000,-0.2230,None,-1.2120,-1.1520,267943000,0.0000,NaN,4358856000,2153504000,2646500000,-116638000,20530000
1,WBA,TRADE & SERVICES,17.240,58.05,41977819000,146172002000,0.0254,5243000000,0.0157,0.0248,28017000000,0.0388,0.175614,91920000000,69014000000,1345000000,1754000000,309000000
2,AAL,ENERGY & TRANSPORTATION,0.998,54.55,12379397000,18685999000,-0.0900,4512000000,-0.3110,-0.5280,-5536000000,0.0303,-0.609089,72464000000,81718000000,18000000000,3470000000,-99000000
3,AMZN,TRADE & SERVICES,13.610,59.47,1742992572000,443297989000,0.0599,59321000000,0.0664,0.0669,152757000000,0.0000,1.168798,360319000000,245516000000,89894000000,12715000000,14288000000
4,SQ,TECHNOLOGY,1.204,74.70,113398292000,15930786000,0.0204,433271000,0.0360,0.0221,2744583000,0.0000,3.289292,13815688000,11078987000,5596137000,395553000,32479000
5,TSLA,MANUFACTURING,18.960,41.37,741720457000,41862001000,0.0435,5751000000,0.0521,0.0776,6630000000,0.0000,1.681861,55146000000,28896000000,16229000000,2124000000,1505000000
6,NSTG,LIFE SCIENCES,1.837,110.25,2559679000,133587000,-0.1280,-67522000,-0.7420,-0.6600,64907000,0.0000,0.020983,513058000,293331000,398018000,-15311000,1112000
7,CRSP,LIFE SCIENCES,12.870,60.40,8943735000,901758000,0.1420,459184000,0.4950,0.4970,-266227000,0.0000,-0.968144,2899519000,295724000,2589446000,807002000,28449000
8,NKE,MANUFACTURING,1.221,83.02,251372552000,44537999000,0.1310,8028000000,0.1290,0.1620,20001000000,0.0067,0.224134,37740000000,24973000000,13476000000,2012000000,174000000
9,AAPL,TECHNOLOGY,0.071,58.49,2462255940000,347155005000,0.1930,110934999000,0.2500,0.2880,104956000000,0.0059,0.184999,329840000000,265560000000,61696000000,21094000000,2093000000


### Conclusion

Hopefully, this demonstration of Alpha Vantage in python proved useful and will enable others easier access to financial data.